In [1]:
import pandas as pd
import numpy as np
import faiss
import collections
import random
from collections import defaultdict
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.python.keras.models import Model
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss,NegativeSampler
import os, math, warnings, math, pickle

path = './data/raw/'
generate = './data/generate/'

/data/ygq/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/ygq/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/ygq/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/ygq/anaconda3/envs/tensorflow/lib/pyt

In [2]:
all_click_df=pd.read_csv(generate+'YGQ.csv')

In [3]:
# 获取双塔召回时的训练验证数据
# negsample指的是通过滑窗构建样本的时候，负样本的数量
def gen_data_set(data, negsample=0):
    data.sort_values("click_timestamp", inplace=True)
    item_ids = data['click_article_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        
        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))   # 用户没看过的文章里面选择负样本
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)  # 对于每个正样本，选择n个负样本
            
        # 长度只有一个的时候，需要把这条数据也放到训练集中，不然的话最终学到的embedding就会有缺失
        if len(pos_list) == 1:
            train_set.append((reviewerID, [pos_list[0]], pos_list[0],1,len(pos_list)))
            test_set.append((reviewerID, [pos_list[0]], pos_list[0],1,len(pos_list)))
            
        # 滑窗构造正负样本
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1])))  # 正样本 [user_id, his_item, pos_item, label, len(his_item)]
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1]))) # 负样本 [user_id, his_item, neg_item, label, len(his_item)]
            else:
                # 将最长的那一个序列长度作为测试数据
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1])))
                
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set

# 将输入的数据进行padding，使得序列特征的长度都一致
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "click_article_id": train_iid, "hist_click_article_id": train_seq_pad,
                         "hist_len": train_hist_len}

    return train_model_input, train_label

In [4]:
def mind_u2i_dict(data, topk=20):    
    k_max=3
    sparse_features = ["click_article_id", "user_id"]
    SEQ_LEN = 30 # 用户点击序列的长度，短的填充，长的截断
    
    user_profile_ = data[["user_id"]].drop_duplicates('user_id')
    item_profile_ = data[["click_article_id"]].drop_duplicates('click_article_id')  
    
    # 类别编码
    features = ["click_article_id", "user_id"]
    feature_max_idx = {}
    
    for feature in features:
        lbe = LabelEncoder()
        data[feature] = lbe.fit_transform(data[feature])
        feature_max_idx[feature] = data[feature].max() + 1
    
    # 提取user和item的画像，这里具体选择哪些特征还需要进一步的分析和考虑
    user_profile = data[["user_id"]].drop_duplicates('user_id')
    item_profile = data[["click_article_id"]].drop_duplicates('click_article_id')  
    
    user_index_2_rawid = dict(zip(user_profile['user_id'], user_profile_['user_id']))
    item_index_2_rawid = dict(zip(item_profile['click_article_id'], item_profile_['click_article_id']))
    
    # 划分训练和测试集
    # 由于深度学习需要的数据量通常都是非常大的，所以为了保证召回的效果，往往会通过滑窗的形式扩充训练样本
    train_set, test_set = gen_data_set(data, 0)
    # 整理输入数据，具体的操作可以看上面的函数
    train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
    test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
    
    # 确定Embedding的维度
    embedding_dim = 16
    
    # 将数据整理成模型可以直接输入的形式
    user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
                            VarLenSparseFeat(SparseFeat('hist_click_article_id', feature_max_idx['click_article_id'], embedding_dim,
                                                        embedding_name="click_article_id"), SEQ_LEN, 'mean', 'hist_len'),]
    item_feature_columns = [SparseFeat('click_article_id', feature_max_idx['click_article_id'], embedding_dim)]
    
    # 模型的定义 
    # num_sampled: 负采样时的样本数量
    
    from collections import Counter

    train_counter = Counter(train_model_input['click_article_id'])
    item_count = [train_counter.get(i, 0) for i in range(item_feature_columns[0].vocabulary_size)]
    sampler_config = NegativeSampler('frequency', num_sampled=5, item_name='click_article_id', item_count=item_count)
    model = MIND(user_feature_columns, item_feature_columns, dynamic_k=False,p=1,k_max=k_max, user_dnn_hidden_units=(128,64, embedding_dim),sampler_config=sampler_config)
    # 模型编译
    model.compile(optimizer="adam", loss=sampledsoftmaxloss)  
    
    # 模型训练，这里可以定义验证集的比例，如果设置为0的话就是全量数据直接进行训练
    history = model.fit(train_model_input, train_label, batch_size=256, epochs=1, verbose=1, validation_split=0.0)
    
    # 训练完模型之后,提取训练的Embedding，包括user端和item端
    test_user_model_input = test_model_input
    all_item_model_input = {"click_article_id": item_profile['click_article_id'].values}

    user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
    item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)
    
    # 保存当前的item_embedding 和 user_embedding 排序的时候可能能够用到，但是需要注意保存的时候需要和原始的id对应
    user_embs = user_embedding_model.predict(test_user_model_input, batch_size=16)
    item_embs = item_embedding_model.predict(all_item_model_input, batch_size=16)
    
    # embedding保存之前归一化一下
    user_embs = user_embs / np.linalg.norm(user_embs, axis=1, keepdims=True)
    item_embs = item_embs / np.linalg.norm(item_embs, axis=1, keepdims=True)
    
#     # 将Embedding转换成字典的形式方便查询
#     raw_user_id_emb_dict = {user_index_2_rawid[k]: \
#                                 v for k, v in zip(user_profile['user_id'], user_embs)}
#     raw_item_id_emb_dict = {item_index_2_rawid[k]: \
#                                 v for k, v in zip(item_profile['click_article_id'], item_embs)}
#     # 将Embedding保存到本地
#     pickle.dump(raw_user_id_emb_dict, open(generate + 'user_youtube_emb.pkl', 'wb'))
#     pickle.dump(raw_item_id_emb_dict, open(generate + 'item_youtube_emb.pkl', 'wb'))
    
    # faiss紧邻搜索，通过user_embedding 搜索与其相似性最高的topk个item
    index = faiss.IndexFlatIP(embedding_dim)
    # 上面已经进行了归一化，这里可以不进行归一化了
#     faiss.normalize_L2(user_embs)
#     faiss.normalize_L2(item_embs)
    index.add(item_embs) # 将item向量构建索引
    print(user_embs.shape)
    
    user_recall_items_dict = collections.defaultdict(dict)
    for i in range(k_max):
        sim, idx = index.search(np.ascontiguousarray(user_embs[:,i,:]), topk) # 通过user去查询最相似的topk个item

        for target_idx, sim_value_list, rele_idx_list in tqdm(zip(test_user_model_input['user_id'], sim, idx)):
            target_raw_id = user_index_2_rawid[target_idx]
            # 从1开始是为了去掉商品本身, 所以最终获得的相似商品只有topk-1
            for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]): 
                rele_raw_id = item_index_2_rawid[rele_idx]
                try:
                    user_recall_items_dict[target_raw_id][rele_raw_id] =max(user_recall_items_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value,user_recall_items_dict[target_raw_id][rele_raw_id])
                except:
                    user_recall_items_dict[target_raw_id][rele_raw_id] =user_recall_items_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value
            
    user_recall_items_dict = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in user_recall_items_dict.items()}

    pickle.dump(user_recall_items_dict, open(generate + 'mind_u2i_dict.pkl', 'wb'))
    return user_recall_items_dict

In [5]:
# 由于这里需要做召回评估，所以讲训练集中的最后一次点击都提取了出来
mind_u2i_dict(all_click_df, topk=20)

100%|██████████| 10000/10000 [00:01<00:00, 9680.23it/s]


Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'
swc
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Epoch 1/1
35096/35096 [==============================] - 4s 118us/step - loss: 2.5458
(10000, 3, 16)


10000it [00:01, 6082.24it/s]
10000it [00:01, 8258.66it/s]
10000it [00:00, 10263.65it/s]


{41112: [(351282, 6.928162097930908),
  (353847, 6.928154468536377),
  (300923, 6.9281535148620605),
  (300275, 6.9281511306762695),
  (202338, 6.928150653839111),
  (288687, 6.928148508071899),
  (286759, 6.928147554397583),
  (107162, 6.928145885467529),
  (284423, 6.92814564704895),
  (327132, 6.928144216537476),
  (225065, 6.928143262863159),
  (353607, 6.928142070770264),
  (313473, 6.928141117095947),
  (226226, 6.92813777923584),
  (255412, 6.9281325340271),
  (363068, 6.928131341934204),
  (84058, 6.928130388259888),
  (271970, 6.928130388259888),
  (357554, 4.6231369972229),
  (111482, 2.304993152618408)],
 180593: [(353847, 6.927910327911377),
  (300923, 6.927908897399902),
  (288687, 6.927904367446899),
  (286759, 6.927902460098267),
  (225065, 4.6434266567230225),
  (107162, 4.643423795700073),
  (313473, 4.643421649932861),
  (84058, 4.643415212631226),
  (300275, 4.5967488288879395),
  (284423, 4.596747398376465),
  (351282, 4.596745729446411),
  (202338, 4.59674549102783